In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
#Configura el Modelo Base excluyendo la última capa de clasificación
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


2024-11-11 10:45:08.407977: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 19s 0us/step


In [9]:
#Congelar las Capas del Modelo Base para que no se actualicen durante el entrenamiento
for layer in base_model.layers:
    layer.trainable = False

In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce la dimensionalidad
x = Dense(1024, activation='relu')(x)  # Capa densa adicional
predictions = Dense(1, activation='sigmoid')(x)  # Capa final para clasificación binaria (gato/no-gato)

# Crea el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
import zipfile
import os

# Ruta del archivo y el directorio de destino
zip_path = '/home/adrian/Descargas/archive (1).zip'

extract_dir = '/home/adrian/Descargas/cats_vs_noncats'

# Extraer el archivo zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Listar los archivos y directorios para ver cómo están organizados
extracted_files = os.listdir(extract_dir)
extracted_files

['catvnoncat']

In [17]:
catvnoncat_dir = os.path.join(extract_dir, 'catvnoncat')
subdirectories = os.listdir(catvnoncat_dir)
subdirectories

['train_catvnoncat.h5', 'test_catvnoncat.h5']

In [19]:
catvnoncat_dir = os.path.join(extract_dir, 'catvnoncat')

In [21]:
import h5py
import numpy as np

def load_dataset(file_path):
    with h5py.File(file_path, 'r') as h5file:
        images = np.array(h5file['train_set_x'] if 'train_set_x' in h5file else h5file['test_set_x'])
        labels = np.array(h5file['train_set_y'] if 'train_set_y' in h5file else h5file['test_set_y'])
        class_list = np.array(h5file['list_classes'])
    return images, labels, class_list

# Rutas completas de los archivos de entrenamiento y prueba
train_file_path = '/home/adrian/Descargas/cats_vs_noncats/catvnoncat/train_catvnoncat.h5'
test_file_path = '/home/adrian/Descargas/cats_vs_noncats/catvnoncat/test_catvnoncat.h5'

# Cargar el conjunto de datos de entrenamiento y prueba
train_images, train_labels, classes = load_dataset(train_file_path)
test_images, test_labels, _ = load_dataset(test_file_path)

print(f"Training set shape: {train_images.shape}, Labels: {train_labels.shape}")
print(f"Test set shape: {test_images.shape}, Labels: {test_labels.shape}")


Training set shape: (209, 64, 64, 3), Labels: (209,)
Test set shape: (50, 64, 64, 3), Labels: (50,)


In [25]:
# Verificar las etiquetas únicas en el conjunto de entrenamiento y de prueba para confirmar que son 0 y 1
unique_train_labels = np.unique(train_labels)
unique_test_labels = np.unique(test_labels)

print("Etiquetas únicas en el conjunto de entrenamiento:", unique_train_labels)
print("Etiquetas únicas en el conjunto de prueba:", unique_test_labels)

train_images_resized = tf.image.resize(train_images, (224, 224)) / 255.0
test_images_resized = tf.image.resize(test_images, (224, 224)) / 255.0

Etiquetas únicas en el conjunto de entrenamiento: [0 1]
Etiquetas únicas en el conjunto de prueba: [0 1]


In [27]:
history = model.fit(
    train_images_resized, train_labels,
    validation_data=(test_images_resized, test_labels),
    epochs=10,
    batch_size=32
)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 792ms/step - accuracy: 0.5622 - loss: 1.1316 - val_accuracy: 0.3400 - val_loss: 1.5521
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 639ms/step - accuracy: 0.6870 - loss: 0.8895 - val_accuracy: 0.6600 - val_loss: 0.6261
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 646ms/step - accuracy: 0.4054 - loss: 0.8439 - val_accuracy: 0.3400 - val_loss: 1.3130
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 639ms/step - accuracy: 0.6468 - loss: 0.7104 - val_accuracy: 0.6200 - val_loss: 0.6690
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 634ms/step - accuracy: 0.6417 - loss: 0.6606 - val_accuracy: 0.3400 - val_loss: 0.8550
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 647ms/step - accuracy: 0.6158 - loss: 0.6512 - val_accuracy: 0.3400 - val_loss: 0.9871
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 648ms/step - accuracy: 0.6768 - loss: 0.6051 - val_accuracy: 0.6200 - val_loss: 0.6642
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 647ms/step - accuracy: 0.6735 - loss: 0.6393 - val_accuracy: 0.3400 - val_loss:

In [29]:
loss, accuracy = model.evaluate(test_images_resized, test_labels)
print(f'Pérdida: {loss}')
print(f'Precisión: {accuracy}')


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - accuracy: 0.3100 - loss: 1.1468
Pérdida: 1.1097832918167114
Precisión: 0.3400000035762787


In [31]:
from tensorflow.keras.preprocessing import image

def predict_image(img_array):
    img_resized = tf.image.resize(img_array, (224, 224)) / 255.0
    img_resized = np.expand_dims(img_resized, axis=0)  # Añade una dimensión de lote

    prediction = model.predict(img_resized)
    if prediction[0] > 0.5:
        print("Es un gato")
    else:
        print("No es un gato")

# Prueba con una imagen del conjunto de prueba
predict_image(test_images[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 846ms/step
No es un gato


In [33]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

def load_and_prepare_image(img_path):
    # Cargar la imagen y redimensionarla a 224x224
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)  # Convertir la imagen en un array
    img_array = img_array / 255.0  # Normalizar los valores a [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión de lote
    return img_array


In [35]:
def predict_images_in_folder(model, folder_path):
    # Iterar sobre cada archivo en la carpeta
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        
        # Asegurarse de que el archivo es una imagen
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            # Preparar la imagen
            prepared_image = load_and_prepare_image(img_path)

            # Hacer la predicción
            prediction = model.predict(prepared_image)

            # Interpretar el resultado y mostrar el nombre del archivo y la predicción
            if prediction[0] > 0.5:
                print(f"{filename}: Es un gato")
            else:
                print(f"{filename}: No es un gato")


In [47]:
predict_images_in_folder(model, 'img')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
descarga.jpeg: No es un gato
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
gato.jpeg: No es un gato
